# Importing required libraries

In [2]:
import pandas as pd
import numpy as np
import pickle
import random
import os
from glob import glob
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings('ignore')
# To execute a cell line by line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Loading data

In [3]:
dodr_location = '/N/project/optum/data/parquet/dodr_81_202107/'

In [4]:
# Getting all latest dodr files

filenames = os.listdir(dodr_location)
for files in filenames:
    print(files)

dod_lr2021q1.parquet
dod_provider_bridge.parquet
dod_r2019q3.parquet
dod_lr2011q2.parquet
dod_r2007q2.parquet
dod_proc2021q1.parquet
dod_m2011q2.parquet
dod_m2015q4.parquet
dod_lr2008q1.parquet
dod_diag2020q2.parquet
dod_proc2008q4.parquet
dod_proc2016q2.parquet
dod_m2018q4.parquet
dod_lr2007q4.parquet
dod_c2014q3.parquet
dod_c2014q1.parquet
dod_c2017q3.parquet
dod_m2014q2.parquet
dod_diag2008q4.parquet
dod_proc2016q1.parquet
dod_lr2019q3.parquet
dod_proc2017q3.parquet
dod_lr2020q1.parquet
dod_lr2020q4.parquet
dod_lr2012q1.parquet
dod_diag2015q2.parquet
dod_proc2014q4.parquet
dod_diag2016q1.parquet
dod_r2013q2.parquet
dod_proc2017q2.parquet
dod_mbr_enroll_r.parquet
dod_lr2012q3.parquet
dod_r2010q1.parquet
dod_lr2015q4.parquet
dod_m2010q1.parquet
dod_r2018q1.parquet
dod_diag2015q3.parquet
dod_c2019q1.parquet
dod_r2011q1.parquet
dod_c2012q3.parquet
dod_lr2016q3.parquet
dod_m2016q2.parquet
dod_lr2015q2.parquet
dod_c2015q2.parquet
dod_lr2013q1.parquet
dod_r2015q3.parquet
dod_lr2017q4.parqu

# Identifying Patient IDs with Opioid in their prescriptions 

Using RX tables and NDC column

In [5]:
# Filtering all RX files

rx_files = glob(dodr_location + '*dod_r2007*')
rx_files

['/N/project/optum/data/parquet/dodr_81_202107/dod_r2007q2.parquet',
 '/N/project/optum/data/parquet/dodr_81_202107/dod_r2007q4.parquet',
 '/N/project/optum/data/parquet/dodr_81_202107/dod_r2007q3.parquet',
 '/N/project/optum/data/parquet/dodr_81_202107/dod_r2007q1.parquet']

In [6]:
# rx_df = pd.read_parquet(rx_files[0])
# rx_df.shape
# rx_df['PATID'].nunique()

In [7]:
# CMS NDC codes

with open('filter_list_no_star', 'rb') as f:
    op_ndc_codes = pickle.load(f)
    
len(op_ndc_codes)

41520

In [8]:
# Filtering RX files for patient IDs with at least one opioid prescription using CMS NDC codes

def filterOpRxPat(files, op_ndc_codes):
    rx_df = pd.DataFrame()
    op_pat_id = []

    for i in files:
        x = pd.read_parquet(i)
        rx_ndc_codes = x['NDC'].unique()
        rx_op_ndc_codes = [s2 for s1 in op_ndc_codes for s2 in rx_ndc_codes if s1 in s2]
        op_pat_temp = x[x['NDC'].isin(rx_op_ndc_codes)]['PATID'].unique()
        temp = x[x['PATID'].isin(op_pat_temp)]
        print(i)
        print("Shape:", x.shape, temp.shape)
        print("NDC codes:", x['NDC'].nunique(), temp['NDC'].nunique())
        print("Patient count:", x['PATID'].nunique(), temp['PATID'].nunique())
        rx_df = rx_df.append(temp)
        op_pat_id.extend(op_pat_temp)
        del x, temp, op_pat_temp

    return rx_df, op_pat_id


In [9]:
rx_op_df, op_pat_list = filterOpRxPat(rx_files, op_ndc_codes)
rx_op_df.shape
rx_op_df['NDC'].nunique()
rx_op_df['PATID'].nunique()

/N/project/optum/data/parquet/dodr_81_202107/dod_r2007q2.parquet
Shape: (39201711, 34) (9215094, 34)
NDC codes: 26166 19805
Patient count: 7185289 1035553
/N/project/optum/data/parquet/dodr_81_202107/dod_r2007q4.parquet
Shape: (40317373, 34) (9970291, 34)
NDC codes: 27134 20735
Patient count: 7203220 1101357
/N/project/optum/data/parquet/dodr_81_202107/dod_r2007q3.parquet
Shape: (38727543, 34) (9019324, 34)
NDC codes: 26613 20075
Patient count: 7047536 1004523
/N/project/optum/data/parquet/dodr_81_202107/dod_r2007q1.parquet
Shape: (39730026, 34) (9895122, 34)
NDC codes: 26012 19910
Patient count: 7438695 1164096


(38099831, 34)

26460

2962506

In [10]:
39201711 + 40317373 + 38727543 + 39730026

157976653

In [11]:
# % of patients with op prescritption out of total patients

# round((rx_op_df['PATID'].nunique()/rx_df['PATID'].nunique())*100, 2)

round(((9215094 + 9970291 + 9019324 + 9895122)/(39201711 + 40317373 + 38727543 + 39730026))*100, 2)

24.12

In [12]:
# non_op_df = rx_df[~rx_df['NDC'].isin(op_ndc_codes)]
# non_op_df.shape
# non_op_df['NDC'].nunique()
# non_op_df['NDC'].unique()

In [13]:
# non_op_df['BRND_NM'].value_counts()

# Filtering patient IDs with opioid prescriptions in Inpatient data

Using Confinement data and above pat IDs

In [14]:
len(op_pat_list)

4305529

In [15]:
type(op_pat_list)

list

In [16]:
# list(set([x for x in op_pat_list if op_pat_list.count(x) > 1]))

In [17]:
# Filtering all inpatient (confinement) files

inp_files = glob(dodr_location + '*dod_c2007*')
inp_files

['/N/project/optum/data/parquet/dodr_81_202107/dod_c2007q4.parquet',
 '/N/project/optum/data/parquet/dodr_81_202107/dod_c2007q1.parquet',
 '/N/project/optum/data/parquet/dodr_81_202107/dod_c2007q3.parquet',
 '/N/project/optum/data/parquet/dodr_81_202107/dod_c2007q2.parquet']

In [18]:
# Function to filter patients with opioids in prescriptions in inpatient data

def filterOpioidPrescPatInp(files, op_presc_pat):
    
    inp_df = pd.DataFrame()
    for i in files:
        x = pd.read_parquet(i)
        temp = x[x['PATID'].isin(op_presc_pat) == True]
        print(i)
        print("Shape:", x.shape, temp.shape)
        print("Patient count:", x['PATID'].nunique(), temp['PATID'].nunique())
        inp_df = inp_df.append(temp)
        del x, temp
        
    return inp_df


In [19]:
# Filtering patients with opioid prescriptions in inpatient data

op_inp_df = filterOpioidPrescPatInp(inp_files, op_pat_list)
op_inp_df.shape

/N/project/optum/data/parquet/dodr_81_202107/dod_c2007q4.parquet
Shape: (308513, 35) (139620, 35)
Patient count: 262577 115859
/N/project/optum/data/parquet/dodr_81_202107/dod_c2007q1.parquet
Shape: (322815, 35) (144199, 35)
Patient count: 274575 119668
/N/project/optum/data/parquet/dodr_81_202107/dod_c2007q3.parquet
Shape: (298214, 35) (137779, 35)
Patient count: 256331 114957
/N/project/optum/data/parquet/dodr_81_202107/dod_c2007q2.parquet
Shape: (317027, 35) (148865, 35)
Patient count: 269858 122994


(570463, 35)

In [20]:
op_inp_df.head()

,PATID,PAT_PLANID,ADMIT_DATE,CHARGE,COINS,CONF_ID,COPAY,DEDUCT,DIAG1,DIAG2,...,PROV,STD_COST,STD_COST_YR,TOS_CD,ICU_IND,ICU_SURG_IND,MAJ_SURG_IND,MATERNITY_IND,NEWBORN_IND,TOS_EXT
0,33003282019,53013962277,2007-11-19,16868.94,0.00,MOLTNOKZKM4OT,250.0,0.0,78659,42789,...,4774878972,6358.05,2019,FAC_IP.ACUTE,Y,N,N,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
2,33003282261,53056588883,2007-10-11,14436.23,1528.00,LN6RLMZN4OK4N,0.0,400.0,64511,66411,...,4774888919,6775.95,2019,FAC_IP.ACUTE,N,N,N,Y,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
4,33003282417,53009524485,2007-12-12,27332.22,0.00,LTNRLLMO4OK4O,250.0,0.0,2182,78820,...,4774573813,11581.80,2019,FAC_IP.ACUTE,N,N,Y,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
9,33003283085,53011734739,2007-11-27,32819.75,0.00,MZLT6RLMKM4OO,0.0,0.0,34690,29623,...,4774872509,9820.65,2019,FAC_IP.ACUTE,N,N,N,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
10,33003283151,53010260932,2007-11-05,67109.48,1414.39,M4MK4OZ4KM4O4,0.0,0.0,41091,5180,...,4774876318,28029.15,2019,FAC_IP.ACUTE,Y,Y,Y,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE


# Filtering out patients with cancer

In [21]:
# List of procedure codes for cancer 

x = 77261
ids = [x]
for i in range(0,538):
    x+=1
    ids.append(x)
canc_proc_codes = map(str, ids)
canc_proc_codes = list(canc_proc_codes)

In [22]:
# Filter patient with cancer diagnosis from above patients

def filterCancPat(op_inp_df, canc_proc_list):
    canc_pat_list = op_inp_df[op_inp_df['PROC1'].isin(canc_proc_list) | op_inp_df['PROC2'].isin(canc_proc_list) | op_inp_df['PROC3'].isin(canc_proc_list) | op_inp_df['PROC4'].isin(canc_proc_list) | op_inp_df['PROC5'].isin(canc_proc_list)]['PATID'].unique()
    op_presc_no_cancer_df = op_inp_df[~op_inp_df['PATID'].isin(canc_pat_list)]    
    
    return op_presc_no_cancer_df

In [23]:
op_presc_no_canc_df = filterCancPat(op_inp_df, canc_proc_codes)
op_presc_no_canc_df.shape

(570463, 35)

In [24]:
len(op_pat_list)
op_presc_no_canc_df['PATID'].nunique()

4305529

407694

# Filtering opioid harm patients

In [25]:
# ICD9 codes for high opioid risk

opioid_harm_icd9_codes = ["96500","96501","96502","96509","E8500","E8501","E8502","E9350","E9351","E9352","3055","3040","3047","E9350","E9351","E9352","E9401","9701","T400X1A","T401X1A","T403X1A","T402X1A","T507X1A"]
len(opioid_harm_icd9_codes)


23

In [26]:
# ICD10 codes for high opioid risk

opioid_harm_icd10_codes = ['F11.10','F11.120','F11.121','F11.122','F11.129','F11.14','F11.150','F11.151','F11.159','F11.181','F11.182','F11.188','F11.19','F11.20','F11.220','F11.221','F11.222','F11.229','F11.23','F11.24','F11.250','F11.251','F11.259','F11.281','F11.282','F11.288','F11.29','F11.90','F11.920','F11.921','F11.922','F11.929','F11.93','F11.94','F11.950','F11.951','F11.959','F11.981','F11.982','F11.988','F11.99','T40.0X1A','T40.0X1D','T40.0X2A','T40.0X2DT40.0X3A','T40.0X3D','T40.0X4A','T40.0X4D','T40.0X5A','T40.0X5D','T40.1X1A','T40.1X1D','T40.1X2A','T40.1X2D','T40.1X3A','T40.1X3D','T40.1X4A','T40.1X4D','T40.2X1A','T40.2X1D','T40.2X2A','T40.2X2D','T40.2X3A','T40.2X3D','T40.2X4A','T40.2X4D','T40.2X5A','T40.2X5D','T40.3X1A','T40.3X1D','T40.3X2A','T40.3X2D','T40.3X3A','T40.3X3D','T40.3X4A','T40.3X4D','T40.3X5A','T40.3X5D','T40.4X1A','T40.4X1D','T40.4X2A','T40.4X2D','T40.4X3A','T40.4X3D','T40.4X4A','T40.4X4D','T40.4X5A','T40.4X5D','T40.601A','T40.601D','T40.602A','T40.602D','T40.603A','T40.603D','T40.604A','T40.604D','T40.605A','T40.605D','T40.691A','T40.691D','T40.692A','T40.692D','T40.693A','T40.693D','T40.694A','T40.694D','T40.695A','T40.695D']

opioid_harm_icd10_codes1 = []

for i in opioid_harm_icd10_codes:
    x = i.replace('.','')
    opioid_harm_icd10_codes1.append(x)
    
len(opioid_harm_icd10_codes1)


108

In [27]:
# Function to filter patients with opioid diagnosis in Inpatient data

def filterOpHarmPat(op_inp_df, icd9_codes,icd10_codes):
    op_inp_final_df = pd.DataFrame()
    x304 = op_inp_df[op_inp_df['DIAG1'].str.startswith('304') | op_inp_df['DIAG2'].str.startswith('304') | op_inp_df['DIAG3'].str.startswith('304') | op_inp_df['DIAG4'].str.startswith('304') | op_inp_df['DIAG5'].str.startswith('304')]
    x305 = op_inp_df[op_inp_df['DIAG1'].str.startswith('305') | op_inp_df['DIAG2'].str.startswith('305') | op_inp_df['DIAG3'].str.startswith('305') | op_inp_df['DIAG4'].str.startswith('305') | op_inp_df['DIAG5'].str.startswith('305')]
    xICD9 = op_inp_df[op_inp_df['DIAG1'].isin(icd9_codes) | op_inp_df['DIAG2'].isin(icd9_codes) | op_inp_df['DIAG3'].isin(icd9_codes) | op_inp_df['DIAG4'].isin(icd9_codes) | op_inp_df['DIAG5'].isin(icd9_codes)]
    xICD10 = op_inp_df[op_inp_df['DIAG1'].isin(icd10_codes) | op_inp_df['DIAG2'].isin(icd10_codes) | op_inp_df['DIAG3'].isin(icd10_codes) | op_inp_df['DIAG4'].isin(icd10_codes) | op_inp_df['DIAG5'].isin(icd10_codes)]
    op_inp_final_df = op_inp_final_df.append([x304, x305, xICD9, xICD10])  
    del x304, x305, xICD9, xICD10
    
    return op_inp_final_df


In [28]:
op_presc_no_canc_df.head()

,PATID,PAT_PLANID,ADMIT_DATE,CHARGE,COINS,CONF_ID,COPAY,DEDUCT,DIAG1,DIAG2,...,PROV,STD_COST,STD_COST_YR,TOS_CD,ICU_IND,ICU_SURG_IND,MAJ_SURG_IND,MATERNITY_IND,NEWBORN_IND,TOS_EXT
0,33003282019,53013962277,2007-11-19,16868.94,0.00,MOLTNOKZKM4OT,250.0,0.0,78659,42789,...,4774878972,6358.05,2019,FAC_IP.ACUTE,Y,N,N,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
2,33003282261,53056588883,2007-10-11,14436.23,1528.00,LN6RLMZN4OK4N,0.0,400.0,64511,66411,...,4774888919,6775.95,2019,FAC_IP.ACUTE,N,N,N,Y,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
4,33003282417,53009524485,2007-12-12,27332.22,0.00,LTNRLLMO4OK4O,250.0,0.0,2182,78820,...,4774573813,11581.80,2019,FAC_IP.ACUTE,N,N,Y,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
9,33003283085,53011734739,2007-11-27,32819.75,0.00,MZLT6RLMKM4OO,0.0,0.0,34690,29623,...,4774872509,9820.65,2019,FAC_IP.ACUTE,N,N,N,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
10,33003283151,53010260932,2007-11-05,67109.48,1414.39,M4MK4OZ4KM4O4,0.0,0.0,41091,5180,...,4774876318,28029.15,2019,FAC_IP.ACUTE,Y,Y,Y,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE


In [29]:
op_harm_df = filterOpHarmPat(op_presc_no_canc_df, opioid_harm_icd9_codes, opioid_harm_icd10_codes1)
op_harm_df.shape

(44192, 35)

In [30]:
op_harm_df.head()

,PATID,PAT_PLANID,ADMIT_DATE,CHARGE,COINS,CONF_ID,COPAY,DEDUCT,DIAG1,DIAG2,...,PROV,STD_COST,STD_COST_YR,TOS_CD,ICU_IND,ICU_SURG_IND,MAJ_SURG_IND,MATERNITY_IND,NEWBORN_IND,TOS_EXT
70,33003292417,53020608226,2007-10-22,11269.12,0.0,MKRRLTOMKM4O4,1500.0,0.0,2920,30401,...,4774859265,8430.48,2019,FAC_IP.ACUTE,N,N,N,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
151,33003302510,53020786882,2007-12-14,103703.53,0.0,M6T4KKLLKM4OR,0.0,0.0,570,27906,...,4774875137,11745.98,2019,FAC_IP.ACUTE,N,N,N,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
197,33003308877,53012832923,2007-12-03,25670.00,0.0,MK4KTKNRKM4O4,0.0,0.0,65421,64831,...,4771730875,11820.60,2019,FAC_IP.ACUTE,N,N,Y,Y,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
324,33003326309,53050166784,2007-12-18,8798.10,0.0,L6ORKKLK4OK4L,0.0,0.0,566,04111,...,4777621499,10790.78,2019,FAC_IP.ACUTE,N,N,N,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
714,33003376249,53042603793,2007-12-06,19242.80,0.0,LO44ONL64OK4Z,0.0,0.0,5770,30491,...,7777423517,11014.65,2019,FAC_IP.ACUTE,Y,N,N,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE


# Filtering all datasets for opioid harm patients

In [31]:
op_harm_pat_list = op_harm_df['PATID'].unique()
len(op_harm_pat_list)

37657

In [32]:
# Filtering all member enrollment files for opioid harm patients

file = dodr_location + 'dod_mbr_enroll_r.parquet'
mbr_temp = pq.ParquetDataset(file, use_legacy_dataset = False, filters=[('PATID','in',op_harm_pat_list)])
mbr_enroll_df = mbr_temp.read().to_pandas()
mbr_enroll_df.shape

(79363, 16)

In [33]:
mbr_enroll_df.head()

,PATID,PAT_PLANID,ASO,BUS,CDHP,ELIGEFF,ELIGEND,FAMILY_ID,GDR_CD,GROUP_NBR,HEALTH_EXCH,LIS_DUAL,PRODUCT,RACE,STATE,YRDOB
0,33003284598,53054044619,N,COM,3,2007-01-01,2008-11-01,3064453660,M,000501014,0,,POS,W,MO,1953
1,33003284598,53142319187,N,COM,U,2007-01-01,2021-03-31,3055505970,M,000325598,0,,POS,W,MO,1953
2,33003285835,53046016007,Y,COM,3,2007-01-01,2011-06-30,3019287733,F,000185528,0,,IND,W,AZ,1943
3,33003287387,53009700121,N,COM,3,2007-01-01,2008-10-17,3048467928,F,000223897,0,,PPO,W,CO,1969
4,33003287387,53009700121,N,COM,3,2008-10-18,2010-03-31,3048467928,F,000223897,0,,PPO,W,FL,1969


In [34]:
def filterOpioidHarmPat(files, pat_list):
    df = pd.DataFrame()
    for i in files:
        x = pd.read_parquet(i)
        temp = x[x['PATID'].isin(pat_list) == True]
        print(i)
        print("Shape:", x.shape, temp.shape)
        print("Patient count:", x['PATID'].nunique(), temp['PATID'].nunique())
        df = df.append(temp)
        del x
        del temp

    return df


In [35]:
# Filtering all prescription files for opioid harm patients

rx_final = filterOpioidHarmPat(rx_files, op_harm_pat_list)
rx_final.shape
rx_final['PATID'].nunique()


/N/project/optum/data/parquet/dodr_81_202107/dod_r2007q2.parquet
Shape: (39201711, 34) (325733, 34)
Patient count: 7185289 30639
/N/project/optum/data/parquet/dodr_81_202107/dod_r2007q4.parquet
Shape: (40317373, 34) (324844, 34)
Patient count: 7203220 29170
/N/project/optum/data/parquet/dodr_81_202107/dod_r2007q3.parquet
Shape: (38727543, 34) (330653, 34)
Patient count: 7047536 30703
/N/project/optum/data/parquet/dodr_81_202107/dod_r2007q1.parquet
Shape: (39730026, 34) (300479, 34)
Patient count: 7438695 28952


(1281709, 34)

37657

In [36]:
39201711 + 40317373 + 38727543 + 39730026

157976653

In [37]:
# % of patients with op prescritption and opioid harm out of total patients

# round((rx_final['PATID'].nunique()/rx_df['PATID'].nunique())*100, 2)

round((rx_final['PATID'].nunique()/(39201711 + 40317373 + 38727543 + 39730026))*100, 2)

0.02

In [38]:
rx_final['PATID'].nunique()

37657

In [39]:
# Filtering all inpatient (confinement) files for opioid harm patients

inp_final = filterOpioidHarmPat(inp_files, op_harm_pat_list)
inp_final.shape
inp_final['PATID'].nunique()


/N/project/optum/data/parquet/dodr_81_202107/dod_c2007q4.parquet
Shape: (308513, 35) (14683, 35)
Patient count: 262577 11367
/N/project/optum/data/parquet/dodr_81_202107/dod_c2007q1.parquet
Shape: (322815, 35) (14940, 35)
Patient count: 274575 11722
/N/project/optum/data/parquet/dodr_81_202107/dod_c2007q3.parquet
Shape: (298214, 35) (15507, 35)
Patient count: 256331 12007
/N/project/optum/data/parquet/dodr_81_202107/dod_c2007q2.parquet
Shape: (317027, 35) (16049, 35)
Patient count: 269858 12390


(61179, 35)

37657

# Getting all non-opipid patients data (from patients with no cancer)

In [40]:
op_presc_no_canc_df.head()
op_presc_no_canc_df.shape
op_presc_no_canc_df['PATID'].nunique()

,PATID,PAT_PLANID,ADMIT_DATE,CHARGE,COINS,CONF_ID,COPAY,DEDUCT,DIAG1,DIAG2,...,PROV,STD_COST,STD_COST_YR,TOS_CD,ICU_IND,ICU_SURG_IND,MAJ_SURG_IND,MATERNITY_IND,NEWBORN_IND,TOS_EXT
0,33003282019,53013962277,2007-11-19,16868.94,0.00,MOLTNOKZKM4OT,250.0,0.0,78659,42789,...,4774878972,6358.05,2019,FAC_IP.ACUTE,Y,N,N,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
2,33003282261,53056588883,2007-10-11,14436.23,1528.00,LN6RLMZN4OK4N,0.0,400.0,64511,66411,...,4774888919,6775.95,2019,FAC_IP.ACUTE,N,N,N,Y,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
4,33003282417,53009524485,2007-12-12,27332.22,0.00,LTNRLLMO4OK4O,250.0,0.0,2182,78820,...,4774573813,11581.80,2019,FAC_IP.ACUTE,N,N,Y,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
9,33003283085,53011734739,2007-11-27,32819.75,0.00,MZLT6RLMKM4OO,0.0,0.0,34690,29623,...,4774872509,9820.65,2019,FAC_IP.ACUTE,N,N,N,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
10,33003283151,53010260932,2007-11-05,67109.48,1414.39,M4MK4OZ4KM4O4,0.0,0.0,41091,5180,...,4774876318,28029.15,2019,FAC_IP.ACUTE,Y,Y,Y,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE


(570463, 35)

407694

In [41]:
len(op_harm_pat_list)

37657

In [42]:
non_op_no_canc_df = op_presc_no_canc_df[~op_presc_no_canc_df['PATID'].isin(op_harm_pat_list)]
non_op_no_canc_df.shape
non_op_no_canc_df['PATID'].nunique()

(509284, 35)

370037

In [43]:
non_op_harm_pat_list = non_op_no_canc_df['PATID'].unique()
len(non_op_harm_pat_list)

370037

In [44]:
# num_of_patients = 40000

In [45]:
# rand_patient_sample =  random.sample(list(set(non_op_no_canc_df.PATID)), num_of_patients)
# len(rand_patient_sample)

In [46]:
# Filtering all member enrollment files for non-opioid harm patients

file = dodr_location + 'dod_mbr_enroll_r.parquet'
mbr_temp = pq.ParquetDataset(file, use_legacy_dataset = False, filters=[('PATID','in', non_op_harm_pat_list)])
mbr_enrol_rand_df = mbr_temp.read().to_pandas()
mbr_enrol_rand_df.shape
mbr_enrol_rand_df['PATID'].nunique()

(846735, 16)

369980

In [47]:
def filterRandPat(files, pat_list):
    df = pd.DataFrame()
    for i in files:
        x = pd.read_parquet(i)
        temp = x[x['PATID'].isin(pat_list) == True]
        print(i)
        print("Shape:", x.shape, temp.shape)
        print("Patient count:", x['PATID'].nunique(), temp['PATID'].nunique())
        df = df.append(temp)
        del x
        del temp

    return df

In [48]:
# Filtering all prescription files for random patients

rx_rand_df = filterRandPat(rx_files, non_op_harm_pat_list)
rx_rand_df.shape
rx_rand_df['PATID'].nunique()


/N/project/optum/data/parquet/dodr_81_202107/dod_r2007q2.parquet
Shape: (39201711, 34) (3050480, 34)
Patient count: 7185289 303467
/N/project/optum/data/parquet/dodr_81_202107/dod_r2007q4.parquet
Shape: (40317373, 34) (3014083, 34)
Patient count: 7203220 285223
/N/project/optum/data/parquet/dodr_81_202107/dod_r2007q3.parquet
Shape: (38727543, 34) (3040671, 34)
Patient count: 7047536 298636
/N/project/optum/data/parquet/dodr_81_202107/dod_r2007q1.parquet
Shape: (39730026, 34) (2920507, 34)
Patient count: 7438695 294117


(12025741, 34)

370037

In [49]:
# Filtering all inpatient (confinement) files for random patients

inp_rand_df = filterRandPat(inp_files, non_op_harm_pat_list)
inp_rand_df.shape
inp_rand_df['PATID'].nunique()


/N/project/optum/data/parquet/dodr_81_202107/dod_c2007q4.parquet
Shape: (308513, 35) (124937, 35)
Patient count: 262577 104492
/N/project/optum/data/parquet/dodr_81_202107/dod_c2007q1.parquet
Shape: (322815, 35) (129259, 35)
Patient count: 274575 107946
/N/project/optum/data/parquet/dodr_81_202107/dod_c2007q3.parquet
Shape: (298214, 35) (122272, 35)
Patient count: 256331 102950
/N/project/optum/data/parquet/dodr_81_202107/dod_c2007q2.parquet
Shape: (317027, 35) (132816, 35)
Patient count: 269858 110604


(509284, 35)

370037

In [50]:
inp_rand_df.head()

,PATID,PAT_PLANID,ADMIT_DATE,CHARGE,COINS,CONF_ID,COPAY,DEDUCT,DIAG1,DIAG2,...,PROV,STD_COST,STD_COST_YR,TOS_CD,ICU_IND,ICU_SURG_IND,MAJ_SURG_IND,MATERNITY_IND,NEWBORN_IND,TOS_EXT
0,33003282019,53013962277,2007-11-19,16868.94,0.00,MOLTNOKZKM4OT,250.0,0.0,78659,42789,...,4774878972,6358.05,2019,FAC_IP.ACUTE,Y,N,N,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
2,33003282261,53056588883,2007-10-11,14436.23,1528.00,LN6RLMZN4OK4N,0.0,400.0,64511,66411,...,4774888919,6775.95,2019,FAC_IP.ACUTE,N,N,N,Y,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
4,33003282417,53009524485,2007-12-12,27332.22,0.00,LTNRLLMO4OK4O,250.0,0.0,2182,78820,...,4774573813,11581.80,2019,FAC_IP.ACUTE,N,N,Y,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
9,33003283085,53011734739,2007-11-27,32819.75,0.00,MZLT6RLMKM4OO,0.0,0.0,34690,29623,...,4774872509,9820.65,2019,FAC_IP.ACUTE,N,N,N,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
10,33003283151,53010260932,2007-11-05,67109.48,1414.39,M4MK4OZ4KM4O4,0.0,0.0,41091,5180,...,4774876318,28029.15,2019,FAC_IP.ACUTE,Y,Y,Y,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE


# Data preprocessing

In [51]:
mbr_enrol_rand_df.head()

,PATID,PAT_PLANID,ASO,BUS,CDHP,ELIGEFF,ELIGEND,FAMILY_ID,GDR_CD,GROUP_NBR,HEALTH_EXCH,LIS_DUAL,PRODUCT,RACE,STATE,YRDOB
0,33003282019,53013962277,N,COM,3,2007-01-01,2007-12-31,3055454992,F,000704570,0,,HMO,W,NE,1962
1,33003282261,53056588883,Y,COM,3,2007-01-01,2007-12-31,3071092766,F,000242359,0,,POS,B,CA,1976
2,33003282417,53009524485,N,COM,3,2007-01-01,2009-06-30,3042140010,F,000700037,0,,HMO,W,AZ,1952
3,33003282532,53078702657,N,MCR,U,2007-01-01,2008-11-30,3070875594,F,STM055013,0,U,HMO,W,MO,1937
4,33003282641,53020320530,N,COM,3,2007-01-01,2007-02-06,3073327045,F,0002P3104,0,,POS,B,NJ,1968


In [52]:
mbr_enrol_rand_df.dtypes

PATID                   int64
PAT_PLANID              int64
ASO                    object
BUS                    object
CDHP                   object
ELIGEFF        datetime64[ns]
ELIGEND        datetime64[ns]
FAMILY_ID               int64
GDR_CD                 object
GROUP_NBR              object
HEALTH_EXCH            object
LIS_DUAL               object
PRODUCT                object
RACE                   object
STATE                  object
YRDOB                   int64
dtype: object

In [53]:
mbr_enrol_rand_df.columns

Index(['PATID', 'PAT_PLANID', 'ASO', 'BUS', 'CDHP', 'ELIGEFF', 'ELIGEND',
       'FAMILY_ID', 'GDR_CD', 'GROUP_NBR', 'HEALTH_EXCH', 'LIS_DUAL',
       'PRODUCT', 'RACE', 'STATE', 'YRDOB'],
      dtype='object')

In [54]:
# Function to aggregate MEMBER ENROLLMENT data at patient level

def mbrPreprocess(mbr_df, year):
    
    mbr_df['AGE'] = year - mbr_df['YRDOB']
#     mbr_df = mbr_df[mbr_df['AGE'] > 10] # Removing all patients with age less than 10 years
    
    mbr_df['TOTAL_MON_COV'] = round(((mbr_df.ELIGEND - mbr_df.ELIGEFF)/np.timedelta64(1, 'M')),0)
    
    for i in ['GDR_CD', 'RACE', 'STATE']:
        mbr_df[i] = mbr_df[i].fillna('U')
        mbr_df[i] = np.where(mbr_df[i] == '', 'U', mbr_df[i])

    mbr_grp_df = mbr_df.groupby('PATID').agg({'TOTAL_MON_COV':'sum', 'AGE':'mean', 'GDR_CD':'last', 'RACE':'last', 'STATE':'last'})
    mbr_grp_df.reset_index(inplace = True)
    
    return mbr_grp_df


In [55]:
mbr_op_df = mbrPreprocess(mbr_enroll_df, 2007)
mbr_op_df.shape
mbr_op_df.head()

(37652, 6)

,PATID,TOTAL_MON_COV,AGE,GDR_CD,RACE,STATE
0,33003284598,193.0,54.0,M,W,MO
1,33003285835,54.0,64.0,F,W,AZ
2,33003287387,58.0,38.0,F,W,AZ
3,33003288343,88.0,62.0,F,W,CO
4,33003289068,30.0,50.0,M,W,MN


In [56]:
mbr_rand_df = mbrPreprocess(mbr_enrol_rand_df, 2007)
mbr_rand_df.shape
mbr_rand_df.head()

(369980, 6)

,PATID,TOTAL_MON_COV,AGE,GDR_CD,RACE,STATE
0,33003282019,12.0,45.0,F,W,NE
1,33003282261,12.0,31.0,F,B,CA
2,33003282417,30.0,55.0,F,W,AZ
3,33003282532,23.0,70.0,F,W,MO
4,33003282641,78.0,39.0,F,B,NJ


In [57]:
mbr_op_df['AGE'].value_counts()
mbr_rand_df['AGE'].value_counts()

 45.0    1228
 47.0    1180
 48.0    1159
 50.0    1149
 44.0    1126
         ... 
 12.0       1
 5.0        1
 2.0        1
-1.0        1
 6.0        1
Name: AGE, Length: 86, dtype: int64

30.0    8039
33.0    7926
31.0    7903
32.0    7850
36.0    7561
        ... 
6.0      496
7.0      489
9.0      475
8.0      440
10.0     437
Name: AGE, Length: 91, dtype: int64

In [58]:
# Function to aggregate MEMBER ENROLLMENT data at patient level

def mbrPreprocess(mbr_df, year):
    
    mbr_df['AGE'] = year - mbr_df['YRDOB']
    mbr_df = mbr_df[mbr_df['AGE'] > 10] # Removing all patients with age less than 10 years
    
#     mbr_df['TOTAL_MON_COV'] = round(((mbr_df.ELIGEND - mbr_df.ELIGEFF)/np.timedelta64(1, 'M')),0)
    
    for i in ['GDR_CD', 'RACE', 'STATE']:
        mbr_df[i] = mbr_df[i].fillna('U')
        mbr_df[i] = np.where(mbr_df[i] == '', 'U', mbr_df[i])

    mbr_grp_df = mbr_df.groupby('PATID').agg({'TOTAL_MON_COV':'sum', 'AGE':'mean', 'GDR_CD':'last', 'RACE':'last', 'STATE':'last'})
    mbr_grp_df.reset_index(inplace = True)
    
    return mbr_grp_df


In [59]:
mbr_op_df = mbrPreprocess(mbr_enroll_df, 2007)
mbr_op_df.shape
mbr_op_df.head()

(37644, 6)

,PATID,TOTAL_MON_COV,AGE,GDR_CD,RACE,STATE
0,33003284598,193.0,54.0,M,W,MO
1,33003285835,54.0,64.0,F,W,AZ
2,33003287387,58.0,38.0,F,W,AZ
3,33003288343,88.0,62.0,F,W,CO
4,33003289068,30.0,50.0,M,W,MN


In [60]:
mbr_non_op_df = mbrPreprocess(mbr_enrol_rand_df, 2007)
mbr_non_op_df.shape
mbr_non_op_df.head()

(363849, 6)

,PATID,TOTAL_MON_COV,AGE,GDR_CD,RACE,STATE
0,33003282019,12.0,45.0,F,W,NE
1,33003282261,12.0,31.0,F,B,CA
2,33003282417,30.0,55.0,F,W,AZ
3,33003282532,23.0,70.0,F,W,MO
4,33003282641,78.0,39.0,F,B,NJ


In [61]:
mbr_op_df['AGE'].value_counts()
mbr_non_op_df['AGE'].value_counts()

45.0    1228
47.0    1180
48.0    1159
50.0    1149
44.0    1126
        ... 
88.0      15
89.0      12
90.0      11
13.0       3
12.0       1
Name: AGE, Length: 79, dtype: int64

30.0    8039
33.0    7926
31.0    7903
32.0    7850
36.0    7561
        ... 
15.0    1170
14.0     917
13.0     798
12.0     663
11.0     508
Name: AGE, Length: 80, dtype: int64

In [62]:
mbr_op_df.to_csv('Opioid_analytics/Akhila/overdose_prediction/data/mbr_op_2007.csv', index = False)

In [63]:
mbr_non_op_df.to_csv('Opioid_analytics/Akhila/overdose_prediction/data//mbr_non_op_2007.csv', index=False)

In [64]:
mbr_op_df['PATID'].nunique(), mbr_non_op_df['PATID'].nunique()

(37644, 363849)

In [65]:
inp_final.columns

Index(['PATID', 'PAT_PLANID', 'ADMIT_DATE', 'CHARGE', 'COINS', 'CONF_ID',
       'COPAY', 'DEDUCT', 'DIAG1', 'DIAG2', 'DIAG3', 'DIAG4', 'DIAG5',
       'DISCH_DATE', 'DRG', 'DSTATUS', 'ICD_FLAG', 'IPSTATUS', 'LOS', 'POS',
       'PROC1', 'PROC2', 'PROC3', 'PROC4', 'PROC5', 'PROV', 'STD_COST',
       'STD_COST_YR', 'TOS_CD', 'ICU_IND', 'ICU_SURG_IND', 'MAJ_SURG_IND',
       'MATERNITY_IND', 'NEWBORN_IND', 'TOS_EXT'],
      dtype='object')

In [66]:
lu_drg = pd.read_excel('DRG_Lookup_Table.xls') 

# Dropping duplicates to keep only the most recent codes
lu_drg = lu_drg.drop_duplicates('DRG',keep='last')
lu_drg.drop(columns = ['EFF_DATE', 'END_DATE'], inplace = True)
lu_drg.shape
lu_drg

(1190, 2)

,DRG,DESCRIPTION
0,000,INVALID_POSSIBLE_INPUT_ERROR
3,001,HRT TRANSPL OR IMPL HRT ASST SYS W MCC
6,002,HRT TRANSPL OR IMPL HRT ASST SYS WO MCC
9,003,"ECMO, TRACH MV>96H OR PDX EX FMN MAJ OR"
13,004,TRACH MV>96H OR PDX EX F/M/N WO MAJ OR
...,...,...
1897,9420,INVALID_POSSIBLE_INPUT_ERROR
1898,9590,INVALID_POSSIBLE_INPUT_ERROR
1899,9832,INVALID_POSSIBLE_INPUT_ERROR
1900,NONE,NO DRG CODE


In [67]:
def inpPreprocess(inp_df, drg_codes):
    
    inp_df.drop(columns = ['PAT_PLANID', 'ADMIT_DATE', 'CHARGE', 'COINS', 'CONF_ID',
       'COPAY', 'DEDUCT', 'DISCH_DATE', 'DSTATUS', 'IPSTATUS', 'POS',
       'PROV', 'STD_COST_YR', 'TOS_CD', 'NEWBORN_IND', 'TOS_EXT'], inplace = True)
    
    inp_df['DRG'] = np.where(inp_df['DRG'] == '', 'UNK', inp_df['DRG'])
    inp_df1 = pd.merge(inp_df, drg_codes, on = ['DRG'], how = 'left')
    inp_df1.rename(columns = {'DESCRIPTION': 'DRG_DESC'}, inplace=True)
    
    diag_cols = ['DIAG1', 'DIAG2', 'DIAG3', 'DIAG4', 'DIAG5']
    for i in diag_cols:
        inp_df1[i] = np.where(inp_df1[i] == '', '0000', inp_df1[i])
        
    proc_cols = ['PROC1', 'PROC2', 'PROC3', 'PROC4', 'PROC5']
    for i in diag_cols:
        inp_df1[i] = np.where(inp_df1[i] == '', '0000', inp_df1[i])
        
    ind_cols = ['ICU_IND', 'ICU_SURG_IND', 'MAJ_SURG_IND','MATERNITY_IND']
    for i in ind_cols:
        inp_df1[i] = inp_df1[i].replace({'Y': 1, 'N': 0})
    
    return inp_df1

In [68]:
inp_final.head()

,PATID,PAT_PLANID,ADMIT_DATE,CHARGE,COINS,CONF_ID,COPAY,DEDUCT,DIAG1,DIAG2,...,PROV,STD_COST,STD_COST_YR,TOS_CD,ICU_IND,ICU_SURG_IND,MAJ_SURG_IND,MATERNITY_IND,NEWBORN_IND,TOS_EXT
26,33003284598,53054044619,2007-10-09,7234.15,0.00,LZRTTRMO4OK4M,0.0,0.00,78659,4019,...,4777699636,6358.05,2019,FAC_IP.ACUTE,Y,N,N,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
61,33003290698,53054348097,2007-12-11,33412.91,1554.51,MNRK64RNKM4OK,0.0,0.00,55321,9974,...,4777718838,18954.75,2019,FAC_IP.ACUTE,N,N,Y,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
70,33003292417,53020608226,2007-10-22,11269.12,0.00,MKRRLTOMKM4O4,1500.0,0.00,2920,30401,...,4774859265,8430.48,2019,FAC_IP.ACUTE,N,N,N,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
74,33003292982,53050270636,2007-11-19,9763.14,0.00,MOT4ZKTKKM4OT,0.0,0.00,49122,41401,...,4777701380,6402.83,2019,FAC_IP.ACUTE,N,N,N,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE
82,33003294584,53047488599,2007-12-27,8183.01,223.92,MRK6ONKNKM4ON,0.0,1371.67,5770,2721,...,4777540381,8014.73,2019,FAC_IP.ACUTE,N,N,N,N,N,FAC_IP.ACUTE.ACUTE.ACUTE.ACUTE


In [69]:
inp_final[inp_final['DRG'] == '']['DRG'].value_counts()

inp_final[inp_final['DIAG5'] == '']['DIAG5'].value_counts()

    7638
Name: DRG, dtype: int64

    19140
Name: DIAG5, dtype: int64

In [70]:
inp_op_df = inpPreprocess(inp_final, lu_drg)
inp_op_df.shape
inp_op_df.head()

(61179, 20)

,PATID,DIAG1,DIAG2,DIAG3,DIAG4,DIAG5,DRG,ICD_FLAG,LOS,PROC1,PROC2,PROC3,PROC4,PROC5,STD_COST,ICU_IND,ICU_SURG_IND,MAJ_SURG_IND,MATERNITY_IND,DRG_DESC
0,33003284598,78659,4019,3051,53081,496,313,9,1,0000000,0000000,0000000,0000000,0000000,6358.05,1,0,0,0,CHEST PAIN
1,33003290698,55321,9974,51889,49320,5601,354,9,6,5361,0000000,0000000,0000000,0000000,18954.75,0,0,1,0,HERNIA PX EXC ING & FEM W CC
2,33003292417,2920,30401,3051,311,30501,897,9,4,9468,0000000,0000000,0000000,0000000,8430.48,0,0,0,0,"ALC/DRUG ABUS/DEP, WO REHAB THER WO MCC"
3,33003292982,49122,41401,4019,412,V4582,192,9,2,0000000,0000000,0000000,0000000,0000000,6402.83,0,0,0,0,CHRONIC OBS PULM DISEASE WO CC/MCC
4,33003294584,5770,2721,3051,V1581,0000,440,9,2,0000000,0000000,0000000,0000000,0000000,8014.73,0,0,0,0,DIS OF PANCREAS EXC MALIG WO CC/MCC


In [71]:
inp_op_df[inp_op_df['DRG'] == '']['DRG'].value_counts()

inp_op_df[inp_op_df['DIAG5'] == '']['DIAG5'].value_counts()

Series([], Name: DRG, dtype: int64)

Series([], Name: DIAG5, dtype: int64)

In [72]:
inp_op_df.isnull().sum()

PATID            0
DIAG1            0
DIAG2            0
DIAG3            0
DIAG4            0
DIAG5            0
DRG              0
ICD_FLAG         0
LOS              0
PROC1            0
PROC2            0
PROC3            0
PROC4            0
PROC5            0
STD_COST         0
ICU_IND          0
ICU_SURG_IND     0
MAJ_SURG_IND     0
MATERNITY_IND    0
DRG_DESC         0
dtype: int64

In [73]:
inp_non_op_df = inpPreprocess(inp_rand_df, lu_drg)
inp_non_op_df.shape
inp_non_op_df.head()

(509284, 20)

,PATID,DIAG1,DIAG2,DIAG3,DIAG4,DIAG5,DRG,ICD_FLAG,LOS,PROC1,PROC2,PROC3,PROC4,PROC5,STD_COST,ICU_IND,ICU_SURG_IND,MAJ_SURG_IND,MATERNITY_IND,DRG_DESC
0,33003282019,78659,42789,340,32723,33394,313,9,2,0000000,0000000,0000000,0000000,0000000,6358.05,1,0,0,0,CHEST PAIN
1,33003282261,64511,66411,66541,66201,V270,775,9,2,7359,7569,7534,0000000,0000000,6775.95,0,0,0,1,VAG DEL WO COMPLICATING DX
2,33003282417,2182,78820,6271,6259,2449,743,9,2,6841,6564,0039,0000000,0000000,11581.80,0,0,1,0,UTER&ADNX PX/NON-MALIGNANCY WO CC/MCC
3,33003283085,34690,29623,30780,0000,0000,103,9,9,3893,0000000,0000000,0000000,0000000,9820.65,0,0,0,0,HEADACHES WO MCC
4,33003283151,41091,5180,41401,25000,53081,247,9,1,0066,3607,3722,8856,8853,28029.15,1,1,1,0,PERC CVASC PX W DRG-ELUT STNT WO MCC


In [74]:
inp_op_df['PATID'].nunique(), inp_rand_df['PATID'].nunique()

(37657, 370037)

In [75]:
inp_op_df.to_csv('Opioid_analytics/Akhila/overdose_prediction/data/inp_op_2007.csv', index=False)

In [77]:
inp_non_op_df.to_csv('Opioid_analytics/Akhila/overdose_prediction/data/inp_non_op_2007.csv', index=False)